<a href="https://colab.research.google.com/github/dinaldoap/jit-sdp-nn/blob/master/notebook/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import random
from scipy.stats import mstats

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/dinaldoap/jit-sdp-data/master/jenkins.csv')
df.head()

,fix,ns,nd,nf,entropy,la,ld,lt,ndev,age,nuc,exp,rexp,sexp,author_date_unix_timestamp,classification,contains_bug
0,False,7.0,7.0,7.0,2.641604,9.0,9.0,426.428571,100.0,0.000093,1.0,5171.0,30.227271,1472.714286,1555326371,None,False
1,False,7.0,7.0,7.0,2.750000,8.0,8.0,426.428571,100.0,6.314775,2.0,5170.0,29.227271,1471.714286,1555326363,None,False
2,False,1.0,1.0,2.0,0.906580,15.0,44.0,96.000000,4.0,0.034722,2.0,629.0,14.828373,414.000000,1554971763,None,False
3,False,1.0,1.0,1.0,0.000000,0.0,0.0,40.000000,1.0,0.000012,1.0,4.0,3.058824,3.000000,1554969774,None,False
4,False,1.0,2.0,4.0,1.662506,14.0,10.0,67.000000,6.0,21.280683,4.0,3.0,2.058824,2.000000,1554967752,Feature Addition,False


In [3]:
df = df.sample(frac=1)
label_col = 'contains_bug'
features_cols = ['fix', 'ns', 'nd', 'nf', 'entropy', 'la', 'ld', 'lt', 'ndev', 'age', 'nuc', 'exp', 'rexp', 'sexp', 'classification']
x = df[features_cols]
x['fix'] = x['fix'].astype('int')
df_classification = pd.get_dummies(x, columns=['classification'])
x = pd.concat([x, df_classification], axis='columns')
x = x.drop(['classification'], axis='columns')
x = x.values
y = df[label_col]
y = y.astype('category')
y = y.cat.codes
y = y.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [4]:
print(x[:3])
print(y[:3])

[[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 1.00000000e+00 4.00000000e+01
  5.00000000e+00 9.03027141e+02 1.00000000e+00 3.54000000e+02
  1.07083505e+01 1.96000000e+02 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  1.00000000e+00 4.00000000e+01 5.00000000e+00 9.03027141e+02
  1.00000000e+00 3.54000000e+02 1.07083505e+01 1.96000000e+02
  1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.80000000e+01 1.00000000e+00 5.30000000e+01
  1.00000000e+00 1.02440972e+00 1.00000000e+00 4.07100000e+03
  2.79254815e+02 4.40000000e+01 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.80000000e+01
  1.00000000e+00 5.30000000e+01 1.00000000e+00 1.02440972e+00
  1.00000000e+00 4.07100000e+03 2.79254815e+02 4.40000000e+01
  1.00000000e+00 0.00000000e+00 0.000

In [0]:
val_index = int( len(x) * 0.8 )
test_index = int( len(x) * 0.9 )
x_train, y_train = x[:val_index], y[:val_index]
x_val, y_val = x[val_index:test_index], y[val_index:test_index]
x_test, y_test = x[test_index:], y[test_index:]

In [6]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 1.00000000e+00 4.00000000e+01
  5.00000000e+00 9.03027141e+02 1.00000000e+00 3.54000000e+02
  1.07083505e+01 1.96000000e+02 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  1.00000000e+00 4.00000000e+01 5.00000000e+00 9.03027141e+02
  1.00000000e+00 3.54000000e+02 1.07083505e+01 1.96000000e+02
  1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.80000000e+01 1.00000000e+00 5.30000000e+01
  1.00000000e+00 1.02440972e+00 1.00000000e+00 4.07100000e+03
  2.79254815e+02 4.40000000e+01 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.80000000e+01
  1.00000000e+00 5.30000000e+01 1.00000000e+00 1.02440972e+00
  1.00000000e+00 4.07100000e+03 2.79254815e+02 4.40000000e+01
  1.00000000e+00 0.00000000e+00 0.000

In [0]:
mean_train = x_train.mean(axis=0)
std_train = x_train.std(axis=0)

In [8]:
print(mean_train)
print(std_train)

[2.83527282e-01 1.59327644e+00 2.48125162e+00 3.91574864e+00
 7.26402248e-01 5.14172744e+01 2.60072925e+01 7.06101901e+02
 1.88825446e+01 8.25845774e+01 1.92955780e+00 1.74923243e+03
 8.61675769e+01 9.07292368e+02 2.83527282e-01 1.59327644e+00
 2.48125162e+00 3.91574864e+00 7.26402248e-01 5.14172744e+01
 2.60072925e+01 7.06101901e+02 1.88825446e+01 8.25845774e+01
 1.92955780e+00 1.74923243e+03 8.61675769e+01 9.07292368e+02
 2.83527282e-01 1.56865787e-01 3.51693819e-02 4.73028187e-01
 1.57745022e-02 3.56348591e-02]
[4.50710065e-01 1.62355187e+00 6.11580745e+00 2.45395811e+01
 1.11022257e+00 7.14663943e+02 5.31087907e+02 1.05905710e+03
 2.67314279e+01 2.15463261e+02 7.53001427e+00 1.98126550e+03
 8.23179580e+01 1.31428998e+03 4.50710065e-01 1.62355187e+00
 6.11580745e+00 2.45395811e+01 1.11022257e+00 7.14663943e+02
 5.31087907e+02 1.05905710e+03 2.67314279e+01 2.15463261e+02
 7.53001427e+00 1.98126550e+03 8.23179580e+01 1.31428998e+03
 4.50710065e-01 3.63674184e-01 1.84207754e-01 4.99271

In [0]:
x_train = (x_train - mean_train) / std_train
x_val = (x_val - mean_train) / std_train
x_test = (x_test - mean_train) / std_train

In [10]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[ 1.58965325 -0.36541884 -0.2422005  -0.11881819 -0.65428525 -0.07054683
  -0.04708692 -0.62895749 -0.51933419  3.80780723 -0.12344702 -0.70421275
  -0.91668001 -0.54119896  1.58965325 -0.36541884 -0.2422005  -0.11881819
  -0.65428525 -0.07054683 -0.04708692 -0.62895749 -0.51933419  3.80780723
  -0.12344702 -0.70421275 -0.91668001 -0.54119896  1.58965325 -0.43133605
  -0.19092238 -0.94743586 -0.12659907 -0.19222805]
 [ 1.58965325 -0.36541884 -0.2422005  -0.11881819 -0.65428525 -0.04675942
  -0.04708692 -0.61668242 -0.6689708  -0.37853399 -0.12344702  1.1718609
   2.34562716 -0.65685076  1.58965325 -0.36541884 -0.2422005  -0.11881819
  -0.65428525 -0.04675942 -0.04708692 -0.61668242 -0.6689708  -0.37853399
  -0.12344702  1.1718609   2.34562716 -0.65685076  1.58965325 -0.43133605
  -0.19092238 -0.94743586 -0.12659907 -0.19222805]
 [-0.62906801 -0.36541884  5.97120637  1.9594569   4.08311586  0.04839019
   0.09413264 -0.62269285 -0.22006099  0.44855697  5.7198354   1.38182771
   1.055331

In [0]:
x_train, y_train = torch.from_numpy(x_train), torch.from_numpy(y_train)
x_val, y_val = torch.from_numpy(x_val), torch.from_numpy(y_val)
x_test, y_test = torch.from_numpy(x_test), torch.from_numpy(y_test)

In [0]:
train_dataset = data.TensorDataset(x_train, y_train)
val_dataset = data.TensorDataset(x_val, y_val)
test_dataset = data.TensorDataset(x_test, y_test)

train_dataloader = data.DataLoader(train_dataset, batch_size=512, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=32)
test_dataloader = data.DataLoader(test_dataset, batch_size=32)

In [13]:
inputs, targets = next(iter(train_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(val_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(test_dataloader))
print(inputs[:3])
print(targets[:3])

tensor([[-0.6291, -0.3654, -0.2422, -0.0781, -0.2886, -0.0342, -0.0301, -0.6393,
         -0.6316, -0.0083,  0.0094,  1.9244,  1.6085,  2.8477, -0.6291, -0.3654,
         -0.2422, -0.0781, -0.2886, -0.0342, -0.0301, -0.6393, -0.6316, -0.0083,
          0.0094,  1.9244,  1.6085,  2.8477, -0.6291, -0.4313, -0.1909,  1.0555,
         -0.1266, -0.1922],
        [-0.6291, -0.3654, -0.2422, -0.0373,  0.6892, -0.0132, -0.0094,  0.1075,
         -0.5942, -0.3362,  0.1422, -0.0415,  1.2146,  0.4061, -0.6291, -0.3654,
         -0.2422, -0.0373,  0.6892, -0.0132, -0.0094,  0.1075, -0.5942, -0.3362,
          0.1422, -0.0415,  1.2146,  0.4061, -0.6291, -0.4313, -0.1909,  1.0555,
         -0.1266, -0.1922],
        [-0.6291, -0.9814, -0.4057, -0.1596, -0.6543, -0.0719, -0.0490, -0.6667,
         -0.7064, -0.3833, -0.2562, -0.8829, -1.0468, -0.6903, -0.6291, -0.9814,
         -0.4057, -0.1596, -0.6543, -0.0719, -0.0490, -0.6667, -0.7064, -0.3833,
         -0.2562, -0.8829, -1.0468, -0.6903, -0.6291,

In [0]:
class Classifier(nn.Module):
  def __init__(self, epochs, input_size, hidden_size, val_loss=None):
    super(Classifier, self).__init__()
    self.epochs = epochs
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.val_loss = val_loss
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fcout = nn.Linear(hidden_size, 1)
  
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.sigmoid(self.fcout(x))
    return x

In [15]:
classifier = Classifier(epochs=100, input_size=x.shape[1], hidden_size=512)
classifier

Classifier(
  (fc1): Linear(in_features=34, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fcout): Linear(in_features=512, out_features=1, bias=True)
)

In [0]:
filename = 'classifier.cpt'
def save_classifier(classifier):
  checkpoint = {
      'epochs': classifier.epochs,
      'input_size': classifier.input_size,
      'hidden_size': classifier.hidden_size,
      'val_loss': classifier.val_loss,
      'state_dict': classifier.state_dict()
  }
  with open(filename, 'wb') as f:
    torch.save(checkpoint, f)

def load_classifier():
  with open(filename, 'rb') as f:
    checkpoint = torch.load(f)
  classifier = Classifier(checkpoint['epochs'], checkpoint['input_size'], checkpoint['hidden_size'], checkpoint['val_loss'])
  classifier.load_state_dict(checkpoint['state_dict'])
  return classifier

In [0]:
save_classifier(classifier)
classifier = load_classifier()

In [18]:
if torch.cuda.is_available():
  classifier = classifier.cuda()
  inputs = inputs.cuda()

classifier(inputs[:3].float())

tensor([[0.5089],
        [0.5105],
        [0.4651]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [0]:
criterion = nn.BCELoss()
optimizer = optim.Adam(params=classifier.parameters(), lr=0.003)

In [20]:
train_loss = 0
epochs = classifier.epochs
for epoch in range(epochs):
  classifier.train()
  for inputs, targets in train_dataloader:
    if torch.cuda.is_available():
      inputs, targets = inputs.cuda(), targets.cuda()

    outputs = classifier(inputs.float())
    loss = criterion(outputs.squeeze(), targets.float())
    train_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  with torch.no_grad():
    classifier.eval()
    val_loss = 0
    for inputs, targets in val_dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

      outputs = classifier(inputs.float())
      loss = criterion(outputs.squeeze(), targets.float())
      val_loss += loss.item()

    train_loss = train_loss / len(train_dataloader)
    val_loss = val_loss / len(val_dataloader)
    print('Epoch: {}, Train loss: {}, Val loss: {}'.format(epoch, train_loss, val_loss))

    if classifier.val_loss is None or val_loss < classifier.val_loss:
      classifier.val_loss = val_loss
      classifier.epochs = epoch
      save_classifier(classifier)

Epoch: 0, Train loss: 0.4518454851288545, Val loss: 0.4002617338770314
Epoch: 1, Train loss: 0.41929472559789543, Val loss: 0.3944960748286624
Epoch: 2, Train loss: 0.40892091186268803, Val loss: 0.38618648601205724
Epoch: 3, Train loss: 0.39733148086265085, Val loss: 0.3878860126592611
Epoch: 4, Train loss: 0.39061833302917065, Val loss: 0.393912924747718
Epoch: 5, Train loss: 0.3892999176081706, Val loss: 0.3977085557815276
Epoch: 6, Train loss: 0.3841679822559333, Val loss: 0.37690653651952744
Epoch: 7, Train loss: 0.3832928031974591, Val loss: 0.3739520317237628
Epoch: 8, Train loss: 0.3782837001124327, Val loss: 0.37238402468593496
Epoch: 9, Train loss: 0.37417520166394824, Val loss: 0.3731482168169398
Epoch: 10, Train loss: 0.3702915768563114, Val loss: 0.3742051793164329
Epoch: 11, Train loss: 0.36937798523259624, Val loss: 0.36866230792120885
Epoch: 12, Train loss: 0.37039320130707726, Val loss: 0.3644409716913575
Epoch: 13, Train loss: 0.36286138585250677, Val loss: 0.36260179

In [0]:
def calculate_recalls(targets, predictions):
  n_classes = 2
  confusion_matrix, _, _ = np.histogram2d(targets.detach().cpu().numpy(), predictions.squeeze().detach().cpu().numpy(), bins=n_classes)
  return np.diag(confusion_matrix) / (np.sum(confusion_matrix, axis=1))

def gmean(classifier, dataloader):
  with torch.no_grad():
    classifier.eval()
    recalls = np.zeros((2))
    for inputs, targets in dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()
      
      outputs = classifier(inputs.float())
      predictions = torch.round(outputs).int()
      recalls += calculate_recalls(targets, predictions)

    recalls = recalls / len(dataloader)
    return mstats.gmean(recalls)    

In [34]:
classifier = load_classifier()
if torch.cuda.is_available():
  classifier = classifier.cuda()
train_accuracy = gmean(classifier, train_dataloader)
test_accuracy = gmean(classifier, test_dataloader)
print('Epochs: {}'.format(classifier.epochs))
print('Train g-mean: {}'.format(train_accuracy.item()))
print('Test g-mean: {}'.format(test_accuracy.item()))

Epochs: 30
Train g-mean: 0.63715097011243
Test g-mean: 0.6254205116801312
